# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [15]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [16]:
# check your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [17]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [18]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [19]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [20]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1 }""")

#### Set Keyspace

In [21]:
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [22]:
"""
This query creates the table music_library. 
This table is modeled in this way to achieve clustering columns stored in a nested sort order. 
The importance of this is to identify where the data is stored within a partition. 
Select queries will be based on the primary key and the subsequent clustering columns specified.
"""
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(sessionId int, itemInSession int, artist_name varchar, song varchar, song_length decimal, PRIMARY KEY(sessionId, itemInSession))"
session.execute(query)

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (sessionId, itemInSession, artist_name, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [24]:
"""
1. This query is modelled based on the primary key = sessionId and composite keys = itemInSession.
2. This is done to attain unique values on artist_name and songs in the music_library table.
3. The sequence in which columns appear is a reflection of the order and how the data is partitioned within the partitions.
   The select statement is therefore modeled in relation to the data partitions
"""

query = "SELECT artist_name, song, song_length FROM music_library WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist_name, row.song, row.song_length)
    
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [25]:
"""
1. The "CREATE" query models the table artist_library with primary key = userId and 
    composite keys = sessionId, itemInSession and song.
2. This table is modeled in this way to achieve clustering columns stored in a nested sort order and 
    to attain unique values on artist_name and songs in the artist_library table.
3. The sequence in which columns appear is a reflection of the order and how the data is partitioned within the partitions.
    The importance of this is to identify where the data is stored within a partition. 
    Select queries are based on the primary key and the subsequent clustering columns specified.
"""

query = "CREATE TABLE IF NOT EXISTS artist_library"
query = query + "(userId int, sessionId int, itemInSession int, song varchar, artist_name varchar, first_name text, last_name text, PRIMARY KEY(userId, sessionId, itemInSession))"
session.execute(query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_library (userId, sessionId, itemInSession, song, artist_name, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]))

query = "SELECT artist_name, song, first_name, last_name FROM artist_library WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
    for row in rows:
        print(row.artist_name, row.first_name+ ' ' + row.last_name)
    
except Exception as e:
    print(e)  

Down To The Bone Sylvie Cruz
Three Drives Sylvie Cruz
Sebastien Tellier Sylvie Cruz
Lonnie Gordon Sylvie Cruz


In [12]:
"""
1. The "CREATE" query models the table user_library with primary key = song and composite keys = userId.
2. This table is modeled in this way to achieve clustering columns stored in a nested sort order and 
    to attain values on who listened to the song.
3. The sequence in which columns appear is a reflection of the order and how the data is partitioned within the partitions.
    The importance of this is to identify where the data is stored within a partition. 
    Select queries are based on the primary key and the subsequent clustering columns specified.
"""

query = "CREATE TABLE IF NOT EXISTS user_library"
query = query + "(song varchar, userId int, first_name text, last_name text, PRIMARY KEY(song, userId))"
session.execute(query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_library (song, userId, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

query = "SELECT first_name, last_name FROM user_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
    for row in rows:
        print(row.first_name, row.last_name)
    
except Exception as e:
    print(e)       
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [13]:
query = "DROP TABLE music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query2 = "DROP TABLE artist_library"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
query3 = "DROP TABLE user_library"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()